In [1]:
from cassandra.cluster import Cluster
import station_pb2_grpc
import station_pb2
import pandas as pd
import sys
import os
import subprocess
import grpc
from datetime import date

from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType,StringType
from pyspark.sql.functions import udf
from pyspark.sql.functions import corr
from pyspark.sql.functions import col
import matplotlib.pyplot as plt

In [2]:
cluster = Cluster(['p6_db_2', 'p6_db_1', 'p6_db_3'])
session = cluster.connect()

Created Cassandra session


In [9]:
spark = (SparkSession.builder
         .appName("cassandra-connector")
         .config("spark.driver.allowMultipleContexts", "true")
         .config("spark.jars.packages", "com.datastax.spark:spark-cassandra-connector_2.12:3.2.0")
         .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions")
         .config("spark.cassandra.connection.host", "p6_db_3,p6_db_2,p6_db_1")  # Cassandra nodes
         .config("spark.cassandra.connection.port", "9042")  # Cassandra port
         .getOrCreate())

## Part 1

In [10]:
def setup_cassandra_table():
    # TODO: Q1
    raise NotImplementedError()

In [11]:
# Q1 Ans
setup_cassandra_table()
print(session.execute("describe keyspace weather").one().create_statement)
print(session.execute("describe type weather.station_record").one().create_statement)
print(session.execute("describe table weather.stations").one().create_statement)

CREATE KEYSPACE weather WITH replication = {'class': 'SimpleStrategy', 'replication_factor': '3'}  AND durable_writes = true;
CREATE TYPE weather.station_record (
    tmin int,
    tmax int
);
CREATE TABLE weather.stations (
    id text,
    date date,
    name text static,
    record frozen<station_record>,
    PRIMARY KEY (id, date)
) WITH CLUSTERING ORDER BY (date ASC)
    AND additional_write_policy = '99p'
    AND bloom_filter_fp_chance = 0.01
    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
    AND cdc = false
    AND comment = ''
    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}
    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
    AND crc_check_chance = 1.0
    AND default_time_to_live = 0
    AND extensions = {}
    AND gc_grace_seconds = 864000
    AND max_index_interval = 2048
    AND memtable_flush_period_in_

### Adding metadata to table

In [12]:
table = spark.read.option("header", True).csv("/datasets/stations_metadata.csv")
table.printSchema()

root
 |-- ID: string (nullable = true)
 |-- LATITUDE: string (nullable = true)
 |-- LONGITUDE: string (nullable = true)
 |-- ELEVATION: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- GSN FLAG: string (nullable = true)
 |-- HCN/CRN FLAG: string (nullable = true)
 |-- WMO ID: string (nullable = true)



In [13]:
# TODO: Code to insert metadata into weather.stations

In [14]:
# Metadata insert verify
result = session.execute("SELECT COUNT(*) FROM weather.stations")
print(result.one()[0])

1313


In [15]:
def get_tokens(station_id):
    # TODO: Q2
    raise NotImplementedError() 

In [16]:
# Q2 Ans
row_token, vnode_token = get_tokens("USC00470273")
print("Row token:", row_token)
print("Vnode token:", vnode_token)

Row token: -1
Vnode token: -1


## Part 2

In [ ]:
# TODO: Connect to the server
channel = None
stub = None

In [ ]:
def simulate_sensor(sensor_id):
    # TODO: gRPC client simulation
    raise NotImplementedError()

In [ ]:
# gRPC client runner
for station in ["USW00014837", "USR0000WDDG", "USW00014898", "USW00014839"]:
    num_failed = simulate_sensor(station)
    if num_failed > 0:
        print("Failed to write all data for", station)
        break
    
    r = stub.StationMax(station_pb2.StationMaxRequest(station=station))
    if r.error:
        print(f"error {r.error} in getting max temp of {station}")
    else:
        print(f"max temp for {station} is {r.tmax}")

## Part 3

In [21]:
cassandra_df = spark.read \
    .format("org.apache.spark.sql.cassandra") \
    .options(table="stations", keyspace="weather") \
    .load()

In [ ]:
def create_weather_view():
    # TODO: Initialize the weather2022 view
    raise NotImplementedError()

In [ ]:
# Weather2022 verify
create_weather_view()
spark.sql("show tables").show()

In [ ]:
# TODO: Q3


In [ ]:
# TODO: Q4

## Part 4

In [ ]:
# TODO: Q5


In [ ]:
# TODO: Q6

In [ ]:
# TODO: Q7

In [ ]:
spark.stop()
session.shutdown()
cluster.shutdown()